<table>
    <tr>
        <td><img src="images/Labs/V/arch.png" alt="Arch" style="width: 70%;"></td>
        <td><img src="images/Labs/V/latent_space.png" alt="Latent space" style="width: 70%;"></td>
    </tr>
</table>

<img src="images/Labs/V/img_vec.png" alt="ImgVec" width=60%, height=60% />

### С текстами это тоже работает

<img src="images/Labs/V/w2v.png" alt="word2vec" width=400px, height=400px /> $$ король - мужчина + женщина = королева $$

In [2]:
import re
import codecs

import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from collections import Counter
from torch.utils.data import DataLoader, Dataset

In [40]:
# https://github.com/LordRonz/lotr-word-cloud/tree/main/1_The_Fellowship_Of_The_Ring

In [2]:
with codecs.open('The_Fellowship_Of_The_Ring.txt', encoding='utf-8', mode='r') as f:    
    docs = f.readlines()

In [3]:
text = ' '.join(docs)
len(text), text[0:5]

(968312, 'this ')

### Tokenization

In [23]:
# Удаляем все символы кроме a-z, @, и #
text = re.sub(r'[^a-z@# ]', '', text.lower())
#re.sub(r'[^а-яА-ЯёЁ]'. ' ', text)
# Разбиваем на пробелы
tokens = text.split()  # отдельные слова
vocab = set(tokens)

In [24]:
len(tokens), len(vocab)

(187226, 8695)

### Lemmatization

Лемматизация – это процесс, который использует словарь и морфологический анализ, чтобы привести слово к его канонической форме – лемме.

In [5]:
#!pip install pymystem3

In [4]:
from pymystem3 import Mystem

In [39]:
m = Mystem()
lemm_list = m.lemmatize('лемматизируй это')
print(lemm_list)
''.join(lemm_list)

['лемматизировать', ' ', 'это', '\n']


'лемматизировать это\n'

In [43]:
engl_ex = 'the boy’s dogs are different sizes' # => the boy dog be differ size'
lemm_list = m.lemmatize(engl_ex)
''.join(lemm_list) # не работает, используйте nltk

'the boy’s dogs are different sizes\n'

###  bag-of-words

Алгоритмы машинного обучения не могут напрямую работать с сырым текстом, поэтому необходимо конвертировать текст в наборы цифр (векторы). Мешок слов – это популярная и простая техника перевести текст в вектор. Она описывает вхождения каждого слова в текст.

Любая информация о порядке или структуре слов игнорируется. Вот почему это называется МЕШКОМ слов. Эта модель пытается понять, встречается ли знакомое слово в документе, но не знает, где именно оно встречается.

In [49]:
bow = ["I like this movie, it's funny.", "I hate this movie.", "This was awesome. I like it", "Nice one. I love it."]
bow

["I like this movie, it's funny.",
 'I hate this movie.',
 'This was awesome. I like it',
 'Nice one. I love it.']

<img src="images/Labs/V/bow.png" alt="bag-of-words" />

### N-grams

N-грамма — это последовательность из нескольких слов. N указывает на количество элементов и может быть любым. Например, если N равно 1, получаются слова, или униграммы (лат. unus, «один»). При N=2 выходят словосочетания из двух слов — биграммы (лат. bis, «дважды»). Если N=3, то это уже триграммы (лат. tres, **«три»), т. е. из трёх слов.

Как выглядят триграммы предложения **«В сто сорок солнц закат пылал»**?


<img src="images/Labs/V/n_grams.png" alt="N-grams" width=60%, height=60% />

### TF-IDF

Мешок слов учитывает частоту употребления слов. Посмотрим, как часто уникальное слово встречается во всём корпусе и в отдельном его тексте.
Оценка важности слова определяется величиной TF-IDF (от англ. term frequency, «частота терма, или слова»; inverse document frequency, «обратная частота документа, или текста»). То есть TF отвечает за количество упоминаний слова в отдельном тексте, а IDF отражает частоту его употребления во всём корпусе.
Формула TF-IDF такая: $$TFIDF = TF * IDF $$

TF рассчитывается так: $$ TF = \frac{t}{n} $$

где t (от англ. term) — количество употребления слова, а n — общее число слов в тексте.

IDF нужна в формуле, чтобы уменьшить вес слов, наиболее распространённых в любом другом тексте заданного корпуса. IDF зависит от общего числа текстов в корпусе (D) и количества текстов, в которых это слово встречается (d) $$ IDF = log_{10}(\frac{D}{d}) $$

Например, рассмотрим корпус из 20-ти стихотворений. Возьмём первое: в нём 40 слов, нам интересно слово «река». В стихе оно встречается пять раз. Всего в корпусе два стихотворения с «рекой». Рассчитаем TF-IDF для слова «река» в первом стихотворении корпуса.

TF равна: $$ TF = \frac{t}{n} = \frac{5}{40} = 0.125 $$

В двух из двадцати стихотворений встречается слово «река», тогда IDF равна: $$ IDF = log_{10}(\frac{D}{d}) = log_{10}(\frac{20}{2}) = 1 $$

TF-IDF получается такой: $$ TFIDF = TF * IDF = 0.125 * 1 = 0.125 $$

Большая величина TF-IDF говорит об уникальности слова в тексте по отношению к корпусу. Чем чаще оно встречается в конкретном тексте и реже в остальных, тем выше значение TF-IDF.

### Word2vec

https://github.com/AntonSHBK/word2vec_pytorch_realization/blob/main/word2vec.ipynb

https://translated.turbopages.org/proxy_u/en-ru.ru.e42a8eca-67c532fd-35706cf3-74722d776562/https/www.geeksforgeeks.org/implement-your-own-word2vecskip-gram-model-in-python/

#### Dataset

<img src="images/Labs/V/dataset_w2v.png" alt="dataset" />

### Train w2v

<img src="images/Labs/V/train_w2v.jpeg" alt="train" />

### Skip Gram

In [1]:
# Функция для создания словаря и подготовки данных
def prepare_data_skip_gram(text: str, window_size=2):
    # Удаляем все символы кроме a-z, @, и #
    text = re.sub(r'[^a-z@# ]', '', text.lower())    
    # Разбиваем на пробелы
    tokens = text.split()    
    vocab = set(tokens)
    word_to_ix = {word: i for i, word in enumerate(vocab)}

    data = []
    for i in range(len(tokens)):
        for j in range(max(0, i - window_size), min(len(tokens), i + window_size + 1)):
            if i != j:
                data.append((tokens[i], tokens[j]))	
    return data, word_to_ix, len(vocab)

In [3]:
window_size = 2
embedding_dim = 10
epochs = 5
batch_size = 2

In [4]:
# Тестовые данные
data = 'Captures Semantic Relationships: The skip-gram model effectively captures semantic relationships between words. It learns word embeddings that encode similar meanings and associations, allowing for tasks like word analogies and similarity calculations. Handles Rare Words: The skip-gram model performs well even with rare words or words with limited occurrences in the training data. It can generate meaningful representations for such words by leveraging the context in which they appear. Contextual Flexibility: The skip-gram model allows for flexible context definitions by using a window around each target word. This flexibility captures local and global word associations, resulting in richer semantic representations. Scalability: The skip-gram model can be trained efficiently on large-scale datasets due to its simplicity and parallelization potential. It can process vast amounts of text data to generate high-quality word embeddings.'
ngramm_data, word_to_ix, vocab_size = prepare_data_skip_gram(data, window_size)

In [7]:
ngramm_data[:6]

[('captures', 'semantic'),
 ('captures', 'relationships'),
 ('semantic', 'captures'),
 ('semantic', 'relationships'),
 ('semantic', 'the'),
 ('relationships', 'captures')]

In [ ]:
# dataset
class SkipGramDataset(Dataset):
    def __init__(self, data, word_to_ix):
        self.data = [(word_to_ix[center], word_to_ix[context]) for center, context in data]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx][0], dtype=torch.long), torch.tensor(self.data[idx][1], dtype=torch.long)

In [ ]:
# model
class Word2VecSkipGramModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2VecSkipGramModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.out_layer = nn.Linear(embedding_dim, vocab_size)
        self.activation_function = nn.LogSoftmax(dim=-1)

    def forward(self, center_word_idx):
        hidden_layer = self.embeddings(center_word_idx)
        out_layer = self.out_layer(hidden_layer)
        log_probs = self.activation_function(out_layer)
        return log_probs

In [ ]:
loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

### CBOW (continuous bag-of-word model)

In [12]:
def prepare_data_cbow(text: str, window_size=2):
    text = re.sub(r'[^a-z@# ]', '', text.lower())    
    tokens = text.split()    

    vocab = set(tokens)
    word_to_ix = {word: i for i, word in enumerate(vocab)}

    data = []
    for i in range(window_size, len(tokens) - window_size):
        context = [tokens[i - j - 1] for j in range(window_size)] + [tokens[i + j + 1] for j in range(window_size)]
        target = tokens[i]
        data.append((context, target))
    return data, word_to_ix, len(vocab)

In [13]:
window_size = 2
embedding_dim = 10
epochs = 5
batch_size = 2

In [14]:
# Тестовые данные
data = 'Captures Semantic Relationships: The skip-gram model effectively captures semantic relationships between words. It learns word embeddings that encode similar meanings and associations, allowing for tasks like word analogies and similarity calculations. Handles Rare Words: The skip-gram model performs well even with rare words or words with limited occurrences in the training data. It can generate meaningful representations for such words by leveraging the context in which they appear. Contextual Flexibility: The skip-gram model allows for flexible context definitions by using a window around each target word. This flexibility captures local and global word associations, resulting in richer semantic representations. Scalability: The skip-gram model can be trained efficiently on large-scale datasets due to its simplicity and parallelization potential. It can process vast amounts of text data to generate high-quality word embeddings.'
ngramm_data, word_to_ix, vocab_size = prepare_data_cbow(data, window_size)

In [15]:
ngramm_data[:6]

[(['semantic', 'captures', 'the', 'skipgram'], 'relationships'),
 (['relationships', 'semantic', 'skipgram', 'model'], 'the'),
 (['the', 'relationships', 'model', 'effectively'], 'skipgram'),
 (['skipgram', 'the', 'effectively', 'captures'], 'model'),
 (['model', 'skipgram', 'captures', 'semantic'], 'effectively'),
 (['effectively', 'model', 'semantic', 'relationships'], 'captures')]

In [ ]:
# dataset 
class CBOWDataset(Dataset):
    def __init__(self, data, word_to_ix):
        self.contexts = []
        self.targets = []
        for context, target in data:
            indexed_context = [word_to_ix[word] for word in context]
            self.contexts.append(indexed_context)
            self.targets.append(word_to_ix[target])

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        # Возвращаем контекст и центральное слово как пару тензоров
        return torch.tensor(self.contexts[idx], dtype=torch.long), torch.tensor(self.targets[idx], dtype=torch.long)

In [ ]:
# model
class Word2VecCBOWModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2VecCBOWModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.out_layer = nn.Linear(embedding_dim, vocab_size)
        self.activation_function = nn.LogSoftmax(dim=1)

    def forward(self, center_word_idx):
        hidden_layer = torch.mean(self.embeddings(center_word_idx), dim=1)
        out_layer = self.out_layer(hidden_layer)
        log_probs = self.activation_function(out_layer)
        return log_probs

In [ ]:
loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [ ]:
# # Извлекаем векторы слов из модели
embeddings = model.embeddings.weight.data.numpy()
ix_to_word = {i: word for word, i in word_to_ix.items()}
w2v_dict = {ix_to_word[ix]: embeddings[ix] for ix in range(vocab_size)}

### nn.Embedding

In [ ]:
import torch
import torch.nn as nn

In [ ]:
num_embeddings = 10 #
embedding_dim = 3
emb = nn.Embedding(num_embeddings, embedding_dim)

In [ ]:
idx = torch.tensor([0])
a = emb(idx)
idx = torch.tensor([1])
b = emb(idx)
a,b

In [52]:
from collections import Counter

In [ ]:
# This is going to be the dummy sentence :
sentences = "this is the second example showing for the article at gfg. and doing this is actually really fun"
 
words = sentences.split(' ')
 
# create a dictionary
vocab = Counter(words) 
vocab = sorted(vocab, key=vocab.get, reverse=True)
vocab # 'this', 'is', 'the', 'second', 'example', 'showing', ...
vocab_size = len(vocab)
 
# create a word to index dictionary from our Vocab dictionary
word2idx = {word: ind for ind, word in enumerate(vocab)} 
 
encoded_sentences = [word2idx[word] for word in words]
encoded_sentences # [0, 1, 2, 3, 4, 5, 6, 2, 7,...
# assign a value to your embedding_dim
e_dim = 5

In [ ]:
# initialise an Embedding layer from Torch
emb = nn.Embedding(vocab_size, e_dim, padding_idx = 0)
word_vectors = emb(torch.LongTensor(encoded_sentences))
 
#print the word_vectors
print(word_vectors)

In [ ]:
emb.weight

In [ ]:
cos = nn.CosineSimilarity(dim=0, eps=1e-6)

In [ ]:
word_vectors[2]

In [ ]:
input1 = word_vectors[2]
for input2 in word_vectors:
    print(cos(input1, input2))